In [ ]:
import os
import asyncio
import edge_tts
import nest_asyncio
nest_asyncio.apply()

# Папка с главами
chapters_dir = "chapters_txt"
audio_dir = "chapters_mp3"
vtt_dir = "chapters_vtt"

os.makedirs(audio_dir, exist_ok=True)
os.makedirs(vtt_dir, exist_ok=True)

# Голос по умолчанию (можно заменить на любой из SUPPORTED_VOICES)
VOICE = "ru-RU-SvetlanaNeural"
RATE = "+0%"
VOLUME = "+0%"

# Обработка одного файла
async def convert_one(txt_path, mp3_path, vtt_path):
    with open(txt_path, "r", encoding="utf-8") as f:
        text = f.read()

    communicate = edge_tts.Communicate(text=text, voice=VOICE, rate=RATE, volume=VOLUME)
    submaker = edge_tts.SubMaker()
    words = []
    with open(mp3_path, "wb") as audio_file:
        async for chunk in communicate.stream():
            if chunk["type"] == "audio":
                audio_file.write(chunk["data"])
            elif chunk["type"] == "WordBoundary":
                words.append(chunk["text"])

    # Записываем субтитры без времени — просто слова через пробел или построчно
    with open(vtt_path, "w", encoding="utf-8") as f:
        # Можно сохранить просто все слова через пробел:
        f.write(" ".join(words))

# Обработка всех файлов
async def batch_convert():
    files = sorted(f for f in os.listdir(chapters_dir) if f.endswith(".txt"))
    for file in files:
        name = os.path.splitext(file)[0]  # например, Глава_01
        txt_path = os.path.join(chapters_dir, file)
        mp3_path = os.path.join(audio_dir, name + ".mp3")
        vtt_path = os.path.join(vtt_dir, name + ".vtt")
        print(f"🎙️ {file} → 🎧 {mp3_path}, 📝 {vtt_path}")
        await convert_one(txt_path, mp3_path, vtt_path)
# Запускаем
await batch_convert()

print("✅ Все главы озвучены и сохранены в", output_dir)

🎙️ 01.txt → 🎧 chapters_mp3/01.mp3, 📝 chapters_vtt/01.vtt
🎙️ 02.txt → 🎧 chapters_mp3/02.mp3, 📝 chapters_vtt/02.vtt
🎙️ 03.txt → 🎧 chapters_mp3/03.mp3, 📝 chapters_vtt/03.vtt
🎙️ 04.txt → 🎧 chapters_mp3/04.mp3, 📝 chapters_vtt/04.vtt
🎙️ 05.txt → 🎧 chapters_mp3/05.mp3, 📝 chapters_vtt/05.vtt


In [11]:
import os
import re
import shutil
from docx import Document

# Путь к docx-файлу
input_file = "Fantom.docx"  # замените на путь к вашей книге
output_dir = "chapters_txt"

# Очистить папку, если она уже существует
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Загружаем документ
doc = Document(input_file)

chapters = []
current_chapter = []
chapter_num = 0

# Шаблон для определения начала главы
chapter_pattern = re.compile(r"^Глава\s+\d+", re.IGNORECASE)

# Разбиваем по главам
for para in doc.paragraphs:
    text = para.text.strip()
    if not text:
        continue

    if chapter_pattern.match(text):
        if current_chapter:
            chapters.append(current_chapter)
            current_chapter = []
        chapter_num += 1
    current_chapter.append(text)

if current_chapter:
    chapters.append(current_chapter)

# Количество цифр в номере файла, например: 01, 02, ..., 10, 11 и т.д.
digits = len(str(len(chapters)))

# Сохраняем главы
for i, chapter in enumerate(chapters, 1):
    filename = os.path.join(output_dir, f"{i:0{digits}}.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write("\n\n".join(chapter))

print(f"✅ Разделено на {len(chapters)} глав. Файлы сохранены в папке '{output_dir}'")

✅ Разделено на 25 глав. Файлы сохранены в папке 'chapters_txt'


In [6]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.5 MB/s eta 0:00:00


In [5]:
import gradio as gr
import edge_tts
import asyncio
import os
# https://speech.platform.bing.com/consumer/speech/synthesize/readaloud/voices/list?trustedclienttoken=6A5AA1D4EAFF4E9FB37E23D68491D6F4 - там голоса брать. думаю поймете. ShortName короче
SUPPORTED_VOICES = {
    'DmitryNeural-Руский(муж.)': 'ru-RU-DmitryNeural',
    'SvetlanaNeural-Русский(жен.)': 'ru-RU-SvetlanaNeural',
    'OstapNeural-Украинский(муж.)': 'uk-UA-OstapNeural',
    'PolinaNeural-Украинский(жен.)': 'uk-UA-PolinaNeural'
}

# Смена голоса
def changeVoice(voices):
    example = SUPPORTED_VOICES[voices]
    example_file = os.path.join(os.getcwd(), "example/"+example+".wav")
    return example_file

# Преобразование текста в речь
async def textToSpeech(text, voices, rate, volume):
    output_file = "output.mp3"
    voices = SUPPORTED_VOICES[voices]
    if (rate >= 0):
        rates = rate = "+" + str(rate) + "%"
    else:
        rates = str(rate) + "%"
    if (volume >= 0):
        volumes = "+" + str(volume) + "%"
    else:
        volumes = str(volume) + "%"
    communicate = edge_tts.Communicate(text,
                                       voices,
                                       rate=rates,
                                       volume=volumes,
                                       proxy=None)
    await communicate.save(output_file)
    audio_file = os.path.join(os.getcwd(), "/content/output.mp3")
    if (os.path.exists(audio_file)):
        return audio_file
    else:
        raise gr.Error("Преобразование не удалось！")
        return FileNotFoundError


# Сбросить результат конвертации
def clearSpeech():
    output_file = os.path.join(os.getcwd(), "output.mp3")
    if (os.path.exists(output_file)):
        os.remove(output_file)
    return None, None


with gr.Blocks(css="style.css", title="Преобразование текста в речь") as demo:
    gr.Markdown("""
    # Преобразование текста в речь через Microsoft Edge
    """)
    with gr.Row():
        with gr.Column():
            text = gr.TextArea(label="Текст", elem_classes="text-area")
            btn = gr.Button("Сгенерировать", elem_id="submit-btn")
        with gr.Column():
            voices = gr.Dropdown(choices=[
                "DmitryNeural-Руский(муж.)", "SvetlanaNeural-Русский(жен.)", "OstapNeural-Украинский(муж.)", "PolinaNeural-Украинский(жен.)"
            ],
                                 value="DmitryNeural-Руский(муж.)",
                                 label="Голос",
                                 info="Пожалуйста, выберите голос",
                                 interactive=True)

            example = gr.Audio(label="Пример голоса",
                              value="/content/rus-edge-tts-webui/example/ru-RU-DmitryNeural.wav",
                              interactive=False,
                              elem_classes="example")

            voices.change(fn=changeVoice,inputs=voices,outputs=example)
            rate = gr.Slider(-100,
                             100,
                             step=1,
                             value=0,
                             label="Увеличение / уменьшение скорости речи",
                             info="Скорость речи быстрее / медленнее",
                             interactive=True)

            volume = gr.Slider(-100,
                               100,
                               step=1,
                               value=0,
                               label="Увеличение / уменьшение громкости звука",
                               info="Увеличить / уменьшить громкость звука",
                               interactive=True)
            audio = gr.Audio(label="Результат",
                             interactive=False,
                             elem_classes="audio")
            btn.click(fn=textToSpeech,
                      inputs=[text, voices, rate, volume],
                      outputs=[audio])

if __name__ == "__main__":
    demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://60bb20847775575d17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://60bb20847775575d17.gradio.live


In [2]:
!pip install edge-tts
!pip install gradio
!pip install asyncio
!git clone https://github.com/hinaichigo-fox/rus-edge-tts-webui.git

fatal: destination path 'rus-edge-tts-webui' already exists and is not an empty directory.
